# Section 1: Preprocessing
## Behavior Analysis
### Generate trial regressors

In [ ]:
import os
import numpy as np
from pandas import concat, read_csv
from scipy.stats import gamma
def normalize(arr): return (arr - arr.min()) / (arr.max() - arr.min())

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior'
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
threshold = 0.005

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load / Concatenate / Prepare Data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
df = []
for subject in subjects:
    
    ## Load CSV.
    csv = read_csv(os.path.join(root_dir,'%s_msit_data.txt' %subject))
    
    ## Limit columns.
    csv = csv[['SubjID','trial','iaps','DBS','interference','valence','arousal','responseTime','responseCorrect']]
    
    ## Rename columns.
    csv.columns = ['Subject', 'Trial', 'IAPS', 'DBS', 'Interference', 'Valence_Obj', 'Arousal_Obj', 'RT', 'Accuracy']

    ## Load IAPS ratings.
    iaps = read_csv(os.path.join(root_dir,'%s_IAPS_SAM.csv' %subject))
    iaps = iaps[['IAPS_Number','Valence','Arousal']]
    iaps.columns = ['IAPS','Valence_Subj','Arousal_Subj']

    ## Merge. Append.
    csv = csv.merge(iaps, on='IAPS')
    cols = ['Subject', 'Trial', 'IAPS', 'DBS', 'Interference', 'Valence_Obj', 'Arousal_Obj', 
            'Valence_Subj', 'Arousal_Subj', 'RT', 'Accuracy']
    csv = csv[cols]
    df.append(csv)

## Merge data. Sort.
df = concat(df)
df['DBS'] = np.where(df['DBS']=='DBSoff',0,1)
df = df.sort_values(['Subject','DBS','Trial']).reset_index(drop=True)

## Normalize regressors.
df['nsArousal'] = normalize(df.Arousal_Subj)
df['nsValence'] = normalize(df.Valence_Subj)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Determine Trials for Inclusion/Exclusion.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Set missing RTs to NaNs.
df['RT'] = np.where(df.Accuracy==-1, np.nan, df.RT)
df['Accuracy'] = np.where(df.Accuracy==-1, np.nan, df.Accuracy)
df['Missing'] = df.Accuracy.isnull().astype(int)

## Add Error column.
df['Error'] = 1 - df.Accuracy

## Add Post-Error Column.
df['PostError'] = 0
for subject in df.Subject.unique():
    error = df.loc[df.Subject==subject,'Error']
    posterror = np.insert(np.roll(error,1)[1:], 0, 0)
    df.loc[df.Subject==subject,'PostError'] = posterror

## Iteratively detect outliers across subjects by fitting a Gamma distribution.
df['GammaCDF'], df['Outlier'] = 0, 0
for subject in df.Subject.unique():
    
    ## Fit Gamma to reaction time distribution.
    shape, loc, scale = gamma.fit(df.loc[(df.Subject==subject)&(~df.RT.isnull()),'RT'], floc=0)
    
    ## Find outliers given likelihood threshold.
    cdf = gamma.cdf(df.loc[(df.Subject==subject)&(~df.RT.isnull()),'RT'], shape, loc=loc, scale=scale)
    outliers = (cdf < threshold) | (cdf > 1 - threshold)
    
    ## Append information.
    df.loc[(df.Subject==subject)&(~df.RT.isnull()), 'GammaCDF'] += cdf
    df.loc[(df.Subject==subject)&(~df.RT.isnull()), 'Outlier'] += outliers.astype(int)
    
## Generate exclude.
df['Exclude'] = np.where( df[['Missing','Error','PostError','Outlier']].sum(axis=1), 1, 0)
print '%s trials (%0.2f%%) excluded.' %(df.Exclude.sum(), df.Exclude.mean())

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
df.to_csv('%s/afMSIT_group_data.csv' %root_dir, index=False)

## Parcellation
### Make EMOTE Labels

In [ ]:
import os, shutil
import numpy as np
import pylab as plt
from mne import read_label, read_source_spaces, read_surface, set_log_level
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fs_dir = '/space/sophia/2/users/EMOTE-DBS/freesurfs'
subject = 'BRTU'

parc = 'laus250'
label_dir = os.path.join(fs_dir,subject,'label',parc)
out_dir = os.path.join(fs_dir,subject,'label','april2016')

if os.path.isdir(out_dir): shutil.rmtree(out_dir)
os.makedirs(out_dir)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build Left Hemisphere Labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

hemi = 'lh'
rr, _ = read_surface(os.path.join(fs_dir, subject, 'surf', '%s.inflated' %hemi))
src = read_source_spaces(os.path.join(fs_dir, subject, 'bem', '%s-oct-6-src.fif' %subject))[0]

lhdict = {'dlpfc_1-lh':['caudalmiddlefrontal_1', 'caudalmiddlefrontal_5', 'caudalmiddlefrontal_6'],
          'dlpfc_2-lh':['caudalmiddlefrontal_2', 'caudalmiddlefrontal_3', 'caudalmiddlefrontal_4'],
          'dlpfc_3-lh':['rostralmiddlefrontal_2', 'rostralmiddlefrontal_3'],
          'dlpfc_4-lh':['rostralmiddlefrontal_1', 'rostralmiddlefrontal_5'],
          'dlpfc_5-lh':['parstriangularis_2', 'parsopercularis_2'],
          'dlpfc_6-lh':['parsopercularis_3', 'parsopercularis_4'],
          'racc-lh':['rostralanteriorcingulate_1','rostralanteriorcingulate_2'],
          'dacc-lh':['caudalanteriorcingulate_1','caudalanteriorcingulate_2',],
          'pcc-lh':['posteriorcingulate_2','posteriorcingulate_3']}

for k,V in lhdict.iteritems():
    
    label = np.sum([read_label(os.path.join(label_dir,'%s-%s.label' %(v,hemi)), subject=subject) 
                    for v in V])
    n_vert = np.intersect1d(src['vertno'], label.vertices).shape[0]
    print '%s\t%s' %(n_vert,k)
    label.save(os.path.join(out_dir, '%s.label' %k))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Build Right Hemisphere Labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

hemi = 'rh'
rr, _ = read_surface(os.path.join(fs_dir, subject, 'surf', '%s.inflated' %hemi))
src = read_source_spaces(os.path.join(fs_dir, subject, 'bem', '%s-oct-6-src.fif' %subject))[1]

rhdict = {'dlpfc_1-rh':['caudalmiddlefrontal_1', 'caudalmiddlefrontal_2', 'caudalmiddlefrontal_5'],
          'dlpfc_2-rh':['caudalmiddlefrontal_3', 'caudalmiddlefrontal_4'],
          'dlpfc_3-rh':['rostralmiddlefrontal_2', 'rostralmiddlefrontal_3'],
          'dlpfc_4-rh':['rostralmiddlefrontal_1', 'rostralmiddlefrontal_5'],
          'dlpfc_5-rh':['parstriangularis_2', 'parsopercularis_1'],
          'dlpfc_6-rh':['parsopercularis_3', 'parsopercularis_4'],
          'racc-rh':['rostralanteriorcingulate_1','rostralanteriorcingulate_2'],
          'dacc-rh':['caudalanteriorcingulate_1','caudalanteriorcingulate_2','caudalanteriorcingulate_3'],
          'pcc-rh':['posteriorcingulate_2','posteriorcingulate_3']}

for k,V in rhdict.iteritems():
    label = np.sum([read_label(os.path.join(label_dir,'%s-%s.label' %(v,hemi)), subject=subject) 
                    for v in V])
    n_vert = np.intersect1d(src['vertno'], label.vertices).shape[0]
    print '%s\t%s' %(n_vert,k)
    label.save(os.path.join(out_dir, '%s.label' %k))

## Preprocesing 1: Raw Data
### Fixing MEWA: Digitization
Something got way messed up. Here we make MNE knows what is EEG and what is extra points.

NOTE: Copied over one of the original files for MEWA and renamed it MEWA_msit_unmasked_raw.fif

In [ ]:
import os
import numpy as np
from mne.io import Raw
from pandas import read_table

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
raw_file = 'MEWA_msit_unmasked_raw.fif'
out_file = 'MEWA_msit_raw.fif'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare digitizations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load data. Get digitization from raw.
raw = Raw(os.path.join(root_dir,'raw',raw_file),preload=False,verbose=False)
digitization = raw.info['dig']

## The last 101 points are extra. Set them to kind=4.
for d in digitization[-101:]: d['kind'] = 4
    
## Get coordinates for EEG points (excluding ref/EOG).
rr = np.array([d['r'] for d in dig if d['kind']==3])[:-2]

## Get channels
chs = raw.info['chs']

## Update location information. This was a huge pain in the ass to figure out.
## We ignore the first four channels (Triggers, EOG) and the last channel (STI014).
for ch, r in zip(chs[4:-1], rr): ch['loc'][:3] = r 

## Update digitization/chs.
raw.info['dig'] = digitization
raw.info['chs'] = chs
raw.save(os.path.join(root_dir,'raw',out_file), overwrite=True)

### Fixing MEWA: Masking channel jumps
Time windows were manually inspected. This step isn't strictly necessary but seemed to help with EOG projections.

NOTE: Copied over one of the original files for MEWA and renamed it MEWA_msit_unmasked_raw.fif

In [ ]:
import os
import numpy as np
import pylab as plt
from mne.io import Raw, RawArray

## Specify parameters.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
raw_file = 'MEWA_msit_unmasked_raw.fif'

## Load data.
raw = Raw(os.path.join(root_dir,'raw',raw_file),preload=True,verbose=False)

## Get data in matrix form.
data = raw._data

## Get list of usuable channels
ch_info = [(n,ch) for n,ch in enumerate(raw.ch_names)]
good_ch = [(n,ch) for n,ch in ch_info if ch not in raw.info['bads']]
good_ch = np.array(good_ch)[4:-1]

## Make mask.
mask = np.zeros(data.shape[1])
times = [(384,394), (663,669)]
for t1, t2 in times:
    mask[(raw.times >= t1) & (raw.times <= t2)] += 1
mask = mask.astype(bool)

## Apply mask.
for ch in good_ch[:,0].astype(int):
    data[ch,mask] = 0

## Make new array. Save.
raw = RawArray(data, raw.info, first_samp=raw.first_samp)
raw.add_eeg_average_proj()
raw.save(os.path.join(root_dir,'raw','MEWA_msit_raw.fif'), overwrite=True, verbose=False)

### Projections: EOG

In [ ]:
import os
from mne import write_proj
from mne.preprocessing import compute_proj_eog
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
subjects = ['MEWA']

# NOTE: Not all subjects work with EOG channel = EOG. 
# Some require other frontal channels due to concatenation.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
for subj in subjects:    
    print 'Making EOG file for %s.' %subj
    
    ## Load files.
    raw_file  = os.path.join( root_dir, 'raw', '%s_msit_raw.fif' %subj )
    raw = Raw(raw_file, preload=True, verbose=False, add_eeg_ref=False)
    raw.del_proj(0)
    ## Make EOG proj. Save.
    proj, _ = compute_proj_eog(raw, n_eeg = 4, average=True, filter_length='20s',
                               reject=dict(eeg=5e-4), flat=dict(eeg=5e-8),  ch_name='F2', n_jobs=3)
    write_proj(os.path.join( root_dir, 'raw', '%s_msit_eog-proj.fif' %subj ), proj)

### Projections: ECG

In [ ]:
import os
from mne import read_proj, write_proj
from mne.preprocessing import compute_proj_ecg
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT_april2016'
subjects = ['CHDR']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
for subj in subjects:    
    print 'Making ECG file for %s.' %subj
    
    ## Load files.
    raw_file = os.path.join( root_dir, 'raw', '%s_msit_raw.fif' %subj )
    eog_file = os.path.join( root_dir, 'raw', '%s_msit-proj.fif' %subj )
    raw = Raw(raw_file, preload=True, verbose=False)
    eog_proj = read_proj(eog_file)
    raw.add_proj(eog_proj, remove_existing=True)
    raw.apply_proj()
    
    ## Make ECG proj. Save.
    ecg_proj, _ = compute_proj_ecg(raw, n_eeg = 4, h_freq = 35., average=True, filter_length='20s',
                                reject=dict(eeg=5e-4), flat=dict(eeg=5e-8), ch_name='P9', n_jobs=3)
    proj = eog_proj + [ecg for ecg in ecg_proj if ecg['desc'] not in [eog['desc'] for eog in eog_proj]]
    write_proj(os.path.join( root_dir, 'raw', '%s_msit-proj.fif' %subj ), proj)

## Preprocessing 2: Epoching
### Make Forward Solutions

In [ ]:
import os
from mne import read_trans, read_bem_solution, read_source_spaces
from mne import make_forward_solution, write_forward_solution
from mne.io import Raw

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Main loop.
root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT_april2016'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

for subject in subjects:
    
    print 'Making forward solution for %s.' %subject
    
    ## Load files.
    raw = Raw(os.path.join(root_dir, 'raw', '%s_msit_raw.fif' %subject), preload=False, verbose=False)
    trans = read_trans(os.path.join(fs_dir,subject,'mri','T1-neuromag','sets','COR-%s.fif' %subject))
    src = read_source_spaces(os.path.join(fs_dir,subject,'bem','%s-oct-6p-src.fif' %subject), verbose=False)
    bem = read_bem_solution(os.path.join(fs_dir,subject,'bem','%s-5120-5120-5120-bem-sol.fif' %subject), verbose=False)
    
    ## Compute and save forward solution.
    make_forward_solution(raw.info, trans, src, bem, fname=os.path.join(root_dir,'fwd','%s_msit-fwd.fif' %subject),
                          meg=False, eeg=True, mindist=1.0, overwrite=True, n_jobs=3, verbose=False)

print 'Done.'

### Make Epochs

In [ ]:
import os
import numpy as np
from mne import compute_covariance, Epochs, EpochsArray, find_events, read_proj, pick_types, set_log_level
from mne.io import Raw
from pandas import read_csv
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Filtering parameters.
l_freq = 0.5
h_freq = 50
l_trans_bandwidth = l_freq / 2.
h_trans_bandwidth = 1.0
filter_length = '20s'
n_jobs = 3

## Epoching parameters.
event_id = dict( FN=1, FI=2, NN=3, NI=4 )      # Alik's convention, isn't he smart!?
tmin = -1.5                                    # Leave some breathing room.
tmax =  3.4                                    # Trial is 1900ms, leave 1500ms of room.
resp_buffer = 1.5                              # 1500ms on either side of response.
baseline = (-0.5,-0.1)
reject_tmin = -0.5
reject_tmax = 1.9
reject   = dict(eeg=150e-6)
flat     = dict(eeg=5e-7)
detrend = None
decim = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load behavior.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT'
data_file = os.path.join( root_dir, 'behavior', 'afMSIT_group_data.csv' )
df = read_csv(data_file)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load behavior.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for subj in subjects:
    
    print 'Loading data for %s.' %subj

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    # Define paths.
    raw_file  = os.path.join( root_dir, 'raw', '%s_%s_raw.fif' %(subj,task) )
    proj_file = os.path.join( root_dir, 'raw', '%s_%s-proj.fif' %(subj,task) )
    
    # Load data.
    raw = Raw(raw_file,preload=True,verbose=False)
    proj = read_proj(proj_file)
    
    ## Add projections.
    proj = [p for p in proj if 'ref' not in p['desc']]
    raw.add_proj(proj, remove_existing=True)
    raw.add_eeg_average_proj()
    raw.apply_proj()
    print raw.info['projs']
    
    ## Reduce dataframe to subject.
    data = df[df.Subject==subj]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make events.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Identifying events for %s.' %subj,  
    events = find_events(raw, stim_channel='Trig1', output='onset', min_duration=0.25, verbose=False)

    # Error catching.
    if data.shape[0] != events.shape[0]: raise ValueError('Mismatching number of stimulus onsets!')
    print '%s events found.' %events.shape[0]
    
    # Update event identifiers.
    n = 1
    for dbs in [0,1]:
        for cond in [0,1]:
            ix, = np.where((data.DBS==dbs)&(data.Interference==cond))
            events[ix,-1] = n
            n+=1

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Filter
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Applying bandpass filter to raw [%s, %s].' %(l_freq, h_freq)
    
    Fs = raw.info['sfreq']
    raw.filter(l_freq = l_freq, h_freq = h_freq, filter_length=filter_length, n_jobs=n_jobs,
               l_trans_bandwidth=l_trans_bandwidth, h_trans_bandwidth=h_trans_bandwidth)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make stimulus-locked epochs.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    # Build initial epochs object.
    picks = pick_types(raw.info, meg=False, eeg=True, exclude='bads')
    epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=baseline, picks=picks,
                    reject=reject, flat=flat, reject_tmin=reject_tmin, reject_tmax=reject_tmax, 
                    proj=True, detrend=detrend, decim=decim)
    
    # First round of rejections.
    epochs.drop_bad()                                                       # Remove bad epochs.
    copy = data.ix[[True if not log else False for log in epochs.drop_log]] # Update CSV based on rejections.
    
    '''NOTE: Making a new dataframe copy is just a shortcut for easy indexing between the Pandas 
       DataFrame and the Epochs object. This is due to the three rounds of rejections being 
       applied to the data (e.g. amplitude, behavior exclusion, equalization).'''
    
    # Drop epochs based on behavior.
    epochs.drop(copy.Exclude.astype(bool))
    
    data = data.ix[[True if not log else False for log in epochs.drop_log]]
    print '%s trials remain after rejections.' %(len(epochs))
    print epochs

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make Response-locked epochs.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Making response-locked epochs.'

    # Build response-locked events.
    response_indices = raw.time_as_index(0.4 + data.RT)             # Compensating for MSIT-lock.
    response_events  = epochs.events.copy()
    response_events[:,0] = response_events[:,0] + response_indices

    # Get data.
    arr = epochs.get_data()
    times = epochs.times

    # Calculate lengths of response-locked epochs.
    response_times = data.RT + 0.4                                  # Compensating for MSIT-lock.
    response_windows = np.array([response_times-resp_buffer, response_times+resp_buffer]).T

    # Iteratively build epochs array.
    trials = []
    for n in xrange(len(epochs)):
        mask = (times >= response_windows[n,0]) & (times <= response_windows[n,1])
        trials.append( arr[n,:,mask] )
    trials = np.array(trials).swapaxes(1,2)

    # Finally, make epochs objects.
    resp_epochs = EpochsArray(trials, epochs.info, response_events, tmin=-resp_buffer, event_id=event_id,)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Saving epoch files.'
    epochs.save(os.path.join(root_dir,'ave','%s_%s_%s_stim-epo.fif' %(subj,task,h_freq)))
    resp_epochs.save(os.path.join(root_dir,'ave','%s_%s_%s_resp-epo.fif' %(subj,task,h_freq)))
    data.to_csv(os.path.join(root_dir,'ave','%s_%s_%s-epo.csv' %(subj,task,h_freq)), index=False)
    
    print '\n#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#\n'
    
print 'Done.'

### Make Covariance Matrices / Inverse Solutions / Morph Maps

In [ ]:
import os
from mne import EpochsArray, read_epochs, read_forward_solution, set_log_level
from mne import compute_covariance, write_cov
from mne import compute_morph_matrix, read_source_spaces
from mne.filter import low_pass_filter
from mne.minimum_norm import make_inverse_operator, write_inverse_operator
from scipy.io import savemat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Analysis parameters.
fmax = 50

## Source localization parameters.
loose = 0.2
depth = 0.8

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))

for subject in subjects:
    
    print 'Processing %s' %subject
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load files.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Load in files.
    epo_file = os.path.join(root_dir,'ave','%s_msit_%s_stim-epo.fif' %(subject,fmax))
    epochs = read_epochs(epo_file, verbose=False)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Secondary objects.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    fwd = read_forward_solution(os.path.join(root_dir, 'fwd', '%s_%s-fwd.fif' %(subject,task)), 
                                surf_ori=True, verbose=False)
    
    ## Compute/save noise covariance matrix & inverse operator.
    noise_cov = compute_covariance(epochs, tmin=-0.5, tmax=0.0, method='shrunk', n_jobs=1)
    write_cov(os.path.join(root_dir,'cov','%s_%s_%s-cov.fif' %(subject,task,h_freq)), noise_cov)
    inv = make_inverse_operator(epochs.info, fwd, noise_cov, loose=loose, depth=depth, verbose=False)
    write_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)), inv)

    ## Pre-compute morph matrix.
    vertices_from = [inv['src'][n]['vertno'] for n in xrange(2)]
    vertices_to = [src[n]['vertno'] for n in xrange(2)]
    morph_mat = compute_morph_matrix(subject, 'fsaverage', vertices_from=vertices_from, 
                                     vertices_to=vertices_to,subjects_dir=fs_dir, smooth=25)
    savemat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject),
            mdict=dict(morph_mat=morph_mat))
    
print 'Done.'